### Documentación

Autor:  Julian David Fernandez
Curso:  Data Engineer - Coderhouse
Fecha:  6 Abril - 2023
Ver  : 1.0.0

#### Proposito

Creación de una ETL, para extracción de informaciíon climatica (ultimos 8 dias) de todos los aeropuertos comerciales del estado de la Floria en los Estados unidos. La informaciíon geografica se encuentra complementada con la información geografica del aeropuerto.

URL API: https://docs.aviationapi.com/

In [173]:
# Importar las librerias
import pandas as pd
import numpy as np
import sqlalchemy
import requests
import psycopg2

### Paso 1 - EXTRACT

Listado de aeropuertos a consultar

#### Pregunta:
1. ¿Cual es la información geografica y administrativa de los aeropuertos localizados en el estado de la florida en USA?

In [174]:
# inicializar lista de lista con los nombres de los aeropuertos de la florida y los codigos FAA y ICAO
data = [['Daytona Beach',	'DAB',	'KDAB'],
        #['Fort Lauderdale',	'FLL',	'KFLL'],
        ['Fort Myers',	'RSW',	'KRSW'],['Fort Walton Beach',	'VPS',	'KVPS'],['Gainesville',	'GNV',	'KGNV'],['Jacksonville',	'JAX',	'KJAX'],['Key West',	'EYW',	'KEYW'],['Melbourne',	'MLB',	'KMLB'],['Miami',	'MIA',	'KMIA'],['Orlando',	'MCO',	'KMCO'],['Panama City',	'ECP',	'KECP'],['Pensacola',	'PNS',	'KPNS'],['Punta Gorda',	'PGD',	'KPGD'],['Sanford',	'SFB',	'KSFB'],['Sarasota',	'SRQ',	'KSRQ'],['St. Petersburg',	'PIE',	'KPIE'],['Tallahassee',	'TLH',	'KTLH'],['Tampa',	'TPA',	'KTPA'], ['West Palm Beach',	'PBI',	'KPBI']]

# Creacion del dataframe 
df_airports = pd.DataFrame(data, columns=['Name', 'FAA', 'ICAO'])

In [175]:
# Extraccion de datos geograficos: API
get_params = '?apt=' + ",".join(df_airports.FAA.values.tolist())
url_endpoint_geo = 'https://api.aviationapi.com/v1/airports' + get_params
url_endpoint_geo

'https://api.aviationapi.com/v1/airports?apt=DAB,RSW,VPS,GNV,JAX,EYW,MLB,MIA,MCO,ECP,PNS,PGD,SFB,SRQ,PIE,TLH,TPA,PBI'

2. ¿Cual es la información climatica de todos los aeropuertos listados en la pregunta anterior?

In [176]:
# Extraccion de datos climaticos: API
url_endpoint_weather = 'https://api.aviationapi.com/v1/weather/metar' + get_params
url_endpoint_weather

'https://api.aviationapi.com/v1/weather/metar?apt=DAB,RSW,VPS,GNV,JAX,EYW,MLB,MIA,MCO,ECP,PNS,PGD,SFB,SRQ,PIE,TLH,TPA,PBI'

### Paso 2 - TRANSFORM

Extraccion - Consulta de información geografica de los aeropuertos

In [177]:
# Consulta de información geografica por medio de un Request HTTP
headers = {'Accept': 'application/json'}
r_geo = requests.get(url_endpoint_geo, headers=headers)

# Conversión de json a objeto en python 
airport_geo_dict = r_geo.json()

# y luego transformar los diccionarios en DataFrames
temp = []

for k, vals in airport_geo_dict.items():
    temp.append(vals[0])

df_airport_geo = pd.DataFrame(temp)

In [178]:
df_airport_geo.head()

,site_number,type,facility_name,faa_ident,icao_ident,region,district_office,state,state_full,county,...,certification_typedate,customs_airport_of_entry,military_joint_use,military_landing,lighting_schedule,beacon_schedule,control_tower,unicom,ctaf,effective_date
0,03147.*A,AIRPORT,DAYTONA BEACH INTL,DAB,KDAB,ASO,ORL,FL,FLORIDA,VOLUSIA,...,I C S 05/1973,N,N,Y,SS-SR,SS-SR,Y,122.950,,11/04/2021
1,03198.2*A,AIRPORT,SOUTHWEST FLORIDA INTL,RSW,KRSW,ASO,ORL,FL,FLORIDA,LEE,...,I D S 05/1983,N,,Y,SEE RMK,SS-SR,Y,122.950,128.750,11/04/2021
2,03534.*A,AIRPORT,EGLIN AFB/DESTIN-FT WALTON BEACH,VPS,KVPS,ASO,ORL,FL,FLORIDA,OKALOOSA,...,I C S 05/1973,N,Y,N,SS-SR,SS-SR,Y,,,11/04/2021
3,03210.*A,AIRPORT,GAINESVILLE RGNL,GNV,KGNV,ASO,ORL,FL,FLORIDA,ALACHUA,...,I B S 05/1973,N,N,Y,SEE RMK,SS-SR,Y,122.950,119.550,11/04/2021
4,03250.1*A,AIRPORT,JACKSONVILLE INTL,JAX,KJAX,ASO,ORL,FL,FLORIDA,DUVAL,...,I D S 05/1973,N,N,Y,SS-SR,SS-SR,Y,122.950,,11/04/2021


##### Control de Cambio - Segunda Entrega

Lista de transformaciones en la data:

1. En el campo site_number eliminar los simbolos de asteriscos
2. En el campo facility_name realizar el proceso de convertir todas las letras a minisculas y la primera en mayuscula de los nombres de los aeropuertos para mejorar la lectura visual.
3. En los campos state_full y county tambien realizar el proceso de convertir todas las letras a minisculas y la primera en mayuscula.
4. Convertir los campos N/Y en tipo boolean

In [179]:
# Punto No.1
df_airport_geo['site_number'] = df_airport_geo['site_number'].str.replace('*','')

# Punto No.2
df_airport_geo['facility_name'] = df_airport_geo['facility_name'].str.capitalize()

# Punto No.3
df_airport_geo['state_full'] = df_airport_geo['state_full'].str.capitalize()
df_airport_geo['county'] = df_airport_geo['county'].str.capitalize()
# Punto No.4
DictConverYN = {'Y': True, 'N': False}
df_airport_geo['customs_airport_of_entry'] = df_airport_geo['customs_airport_of_entry'].map(DictConverYN) # Remplazar N/Y a Boolean True/False
df_airport_geo['military_joint_use'] = df_airport_geo['military_joint_use'].map(DictConverYN) # Remplazar N/Y a Boolean True/False
df_airport_geo['military_joint_use'] = df_airport_geo['military_joint_use'].fillna(False)

df_airport_geo['military_landing'] = df_airport_geo['military_landing'].map(DictConverYN) # Remplazar N/Y a Boolean True/False
df_airport_geo['control_tower'] = df_airport_geo['control_tower'].map(DictConverYN) # Remplazar N/Y a Boolean True/False

<ipython-input-179-f762b0a4d9cf>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_airport_geo['site_number'] = df_airport_geo['site_number'].str.replace('*','')


In [180]:
df_airport_geo.head()

,site_number,type,facility_name,faa_ident,icao_ident,region,district_office,state,state_full,county,...,certification_typedate,customs_airport_of_entry,military_joint_use,military_landing,lighting_schedule,beacon_schedule,control_tower,unicom,ctaf,effective_date
0,03147.A,AIRPORT,Daytona beach intl,DAB,KDAB,ASO,ORL,FL,Florida,Volusia,...,I C S 05/1973,False,False,True,SS-SR,SS-SR,True,122.950,,11/04/2021
1,03198.2A,AIRPORT,Southwest florida intl,RSW,KRSW,ASO,ORL,FL,Florida,Lee,...,I D S 05/1983,False,False,True,SEE RMK,SS-SR,True,122.950,128.750,11/04/2021
2,03534.A,AIRPORT,Eglin afb/destin-ft walton beach,VPS,KVPS,ASO,ORL,FL,Florida,Okaloosa,...,I C S 05/1973,False,True,False,SS-SR,SS-SR,True,,,11/04/2021
3,03210.A,AIRPORT,Gainesville rgnl,GNV,KGNV,ASO,ORL,FL,Florida,Alachua,...,I B S 05/1973,False,False,True,SEE RMK,SS-SR,True,122.950,119.550,11/04/2021
4,03250.1A,AIRPORT,Jacksonville intl,JAX,KJAX,ASO,ORL,FL,Florida,Duval,...,I D S 05/1973,False,False,True,SS-SR,SS-SR,True,122.950,,11/04/2021


In [181]:
# Consulta de información climatica por medio de un Request HTTP
r_weather = requests.get(url_endpoint_weather, headers=headers)

# Conversión de json a objeto en python 
airport_weather_dict = r_weather.json()

In [171]:

# y luego transformar los diccionarios en DataFrames
temp = []

# ajuste del objeto json con el objetivo de transformar todos los niveles del objeto en una tabla columnar.
for k, vals in airport_weather_dict.items():

    try:
        sky_conditions = vals['sky_conditions'][0]
        sky_conditions["sky_conditionscoverage"] = sky_conditions.pop("coverage")
        sky_conditions["sky_conditionsbase_agl"] = sky_conditions.pop("base_agl")
    except:
        sky_conditions = 0
    
    vals.update(sky_conditions)
    temp.append(vals)

df_airport_weather = pd.DataFrame(temp)

# limpieza de información raw (datos crudos) la cual es redundante para el dataset.
del df_airport_weather['raw']
del df_airport_weather['sky_conditions']

#df_airport_geo['site_number'] = df_airport_geo['site_number'].str.replace('', '0')

In [185]:
df_airport_weather.head()

,station_id,temp,dewpoint,wind,wind_vel,visibility,alt_hg,alt_mb,wx,auto_report,category,report_type,time_of_obs,sky_conditionscoverage,sky_conditionsbase_agl
0,KVPS,20.5,19.3,130,11,10.0,29.74,1008.1,None,true,IFR,METAR,2023-04-27T23:55:00Z,OVC,800
1,KMCO,23.3,22.2,50,11,10.0,29.84,1010.3,-RA,true,VFR,METAR,2023-04-27T23:53:00Z,FEW,2000
2,KMIA,27.8,22.2,160,8,10.0,29.87,1012.0,None,true,VFR,METAR,2023-04-27T23:53:00Z,FEW,2300
3,KTLH,20.0,18.9,170,4,10.0,29.81,1009.3,-TSRA,true,VFR,METAR,2023-04-27T23:53:00Z,FEW,2500
4,KMLB,26.1,18.9,160,14,10.0,29.81,1009.6,VCTS,true,VFR,METAR,2023-04-27T23:53:00Z,BKN,10000


In [186]:
# Cambio de tipos de datos de object (strings) a su respectivo formato
convert_dict = {
                    'temp'       : 'float', 
                    'dewpoint'   : 'float', 
                    'wind'       : 'int', 
                    'wind_vel'   : 'int',
                    'visibility' : 'float', 
                    'alt_hg'     : 'float', 
                    'alt_mb'     : 'float', 
                    'sky_conditionsbase_agl': 'float'
                }

df_airport_weather = df_airport_weather.astype(convert_dict)

### Pase 3 - LOAD

Conjunto de pasos para la carga de datos dentro de Amazon Redshift, carga de la informacion geografica y la información climatica.

In [187]:
# metodo para importar credenciales de acceso desde una archivo .py con las credenciales (seguridad para publicar en GITHUB)

# url       = "xxxxxxxx.redshift.amazonaws.com" 
# port      = "5439"
# data_base = "xxxxx-database" 
# user      = "xxxxx_coderhouse"
# pwd       = "xxxXXXXxxx"
# myschema  = "xxxxx_coderhouse" 

from credentials import *

In [188]:
from sqlalchemy import create_engine

#Se crear la conexión
conn = create_engine(f'postgresql://{user}:{pwd}@{url}:{port}/{data_base}')

In [193]:
# Carga de la información actualizada del clima, se utiliza la opcion append, para tener un historico del clima, cada vez que se ejecuta
# el ETl.
df_airport_weather.to_sql('proyecto_weather', conn, index=False, if_exists='replace', schema=myschema)

In [190]:
# Carga de la información actualizada del aeropuerto, se remplaza la información de los aeropeurtos, por no ser una varible de tiempo
# sino que contiene información descriptiva y de estado de cada aeropuerto.
df_airport_geo.to_sql('proyecto_airports', conn, index=False, if_exists='replace', schema=myschema)

In [191]:
# Se guardan copias locales en archivos de texto CSV, como soporte
df_airport_geo.to_csv('airport_geo.csv', index=False)

In [192]:
# Se guardan copias en archivos de texto CSV, como soporte
df_airport_weather.to_csv('airport_weather.csv', index=False, mode='a', header=False)